<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Custom-functions" data-toc-modified-id="Custom-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Custom functions</a></span></li><li><span><a href="#Importing-data" data-toc-modified-id="Importing-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Importing data</a></span></li><li><span><a href="#Spliting-into-train-and-test-sets" data-toc-modified-id="Spliting-into-train-and-test-sets-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Spliting into train and test sets</a></span></li><li><span><a href="#Data-transformation-and-standarization" data-toc-modified-id="Data-transformation-and-standarization-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Data transformation and standarization</a></span></li></ul></li><li><span><a href="#Training-initial-DNN-model" data-toc-modified-id="Training-initial-DNN-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training initial DNN model</a></span><ul class="toc-item"><li><span><a href="#Base-configuration" data-toc-modified-id="Base-configuration-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Base configuration</a></span></li><li><span><a href="#Adding-Drop-out" data-toc-modified-id="Adding-Drop-out-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Adding Drop-out</a></span></li><li><span><a href="#Adding-batch-normalization" data-toc-modified-id="Adding-batch-normalization-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Adding batch normalization</a></span></li><li><span><a href="#Adding-Leaky-Relu" data-toc-modified-id="Adding-Leaky-Relu-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Adding Leaky Relu</a></span></li></ul></li><li><span><a href="#Expanding-network-architecture-and-learning-rate-tuning" data-toc-modified-id="Expanding-network-architecture-and-learning-rate-tuning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Expanding network architecture and learning rate tuning</a></span><ul class="toc-item"><li><span><a href="#Adding-hidden-layer-with-1024-neurons" data-toc-modified-id="Adding-hidden-layer-with-1024-neurons-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Adding hidden layer with 1024 neurons</a></span></li><li><span><a href="#Adjusting-learning-rate-decay" data-toc-modified-id="Adjusting-learning-rate-decay-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Adjusting learning rate decay</a></span></li><li><span><a href="#Callbacks-stop" data-toc-modified-id="Callbacks-stop-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Callbacks stop</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

In [4]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Preparation

## Import libraries

In [1]:
import pandas as pd
import numpy as np

import glob
import os

from datetime import datetime

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [5]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

## Custom functions

In [6]:
def performance_summary(model, X_test, y_test ):
    
    y_hat=model.predict(X_test)
    
    df_summary=pd.DataFrame(y_hat, columns=["y_hat"])
    df_summary["y_true"]=y_test
    df_summary["abs_error"]=np.abs(df_summary.y_true-df_summary.y_hat)
    df_summary["error"]=df_summary.y_hat-df_summary.y_true
    df_summary["relative_error"]= df_summary["error"]/df_summary.y_true
    df_summary["relative_abs_error"]= df_summary["abs_error"]/df_summary.y_true
    
    share_within_5pct=(df_summary.query("relative_abs_error<0.05").shape[0]/df_summary.shape[0])*100
    
    print("{:.2f}% : Share of forecasts within 5% absolute error\n".format(share_within_5pct))
    print("{:.0f}   : Mean absolute error \n".format(df_summary.abs_error.mean()))
    print("{:.2f}% : Mean absolute percentage error\n".format(df_summary.relative_abs_error.mean()*100))



In [7]:
def DNN_plot_loss(history, starting_epoch,previous_val_loss):

        trace0=go.Scatter(
                y=history.history['loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="blue",
                size=5,
                opacity=0.5
                ),
                name="Training Loss"
            )


        trace1=go.Scatter(
                y=history.history['val_loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="red",
                size=5,
                opacity=0.5
                ),
                name="Validation Loss"
            )
        
        trace2=go.Scatter(
                y=list(np.ones([len(history.epoch[starting_epoch:])])*np.asarray(previous_val_loss).min()),
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="grey",
                size=5,

                ),
                name="Lowest error from previous models"
            )

        data=[trace0, trace1,trace2]
        figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Loss",range=(900,1500)),
                xaxis=dict(title="Epoch",range=(starting_epoch,history.epoch[-1])),
                legend=dict(
                    x=0.57,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None,



            )))
        iplot(figure)

## Importing data

In [8]:
df = pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Project_Portfolio/master/03_Real_Estate_pricing_in_Warsaw/top_features_data.csv")
print("Data dimensions: {}".format(df.shape))

Data dimensions: (24935, 46)


In [9]:
## Extracting X and y from data, removing features not used in regression
y=df.unit_price
X=df.drop(columns=["unit_price","lat_mod","lon_mod"])

## Spliting into train and test sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [13]:
df.columns

Index(['build_year', 'building_floors_num', 'rooms_num',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_balcony', 'Extras_types_garden', 'Extras_types_lift',
       'floor_num', 'east_bank', 'distance_driving', 'distance_transit',
       'time_driving', 'time_transit', 'restaurant_price_level',
       'restaurant_mean_rating', 'restaurant_mean_popularity',
       'restaurant_count', 'restaurant_ratings_count', 'district_Bemowo',
       'district_Bialoleka', 'district_Downtown', 'district_Subburbs',
       'district_Targowek', 'district_Wawer', 'district_Wola',
       'district_Zoliborz', 'market_primary', 'Building_material_brick',
       'Building_ownership_full_ownership', 'Building_type_apartment',
       'Building_type_block', 'Building_type_tenement',
       'Construction_status_ready_to_use', 'Construction_status

## Data transformation and standarization

In [14]:
# Reshaping y
y_train=np.asarray(y_train).reshape(-1,1)
y_test=np.asarray(y_test).reshape(-1,1)

In [15]:
#Standarizing X
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Training initial DNN model

## Base configuration

In [17]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Initial_model",)


In [18]:
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 515,329
Trainable params: 515,329
Non-trainable params: 0
_______________________________________________

In [19]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 2s 123us/sample - loss: 11118.3712 - val_loss: 10586.6312
Epoch 2/200
19948/19948 [==============================] - 0s 15us/sample - loss: 7066.7532 - val_loss: 3763.7942
Epoch 3/200
19948/19948 [==============================] - 0s 8us/sample - loss: 2464.6213 - val_loss: 1684.2397
Epoch 4/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1628.4329 - val_loss: 1496.9290
Epoch 5/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1463.1656 - val_loss: 1407.6186
Epoch 6/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1387.9316 - val_loss: 1362.7425
Epoch 7/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1347.6677 - val_loss: 1336.8397
Epoch 8/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1319.9563 - val_loss: 1317.9443
Epoch 9/200
19948/19948 [=================

19948/19948 [==============================] - 0s 7us/sample - loss: 1044.2532 - val_loss: 1175.5885
Epoch 73/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1039.1478 - val_loss: 1179.1760
Epoch 74/200
19948/19948 [==============================] - 0s 6us/sample - loss: 1038.9988 - val_loss: 1184.7684
Epoch 75/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1045.3983 - val_loss: 1178.2782
Epoch 76/200
19948/19948 [==============================] - 0s 6us/sample - loss: 1030.0149 - val_loss: 1169.4896
Epoch 77/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1024.2890 - val_loss: 1168.7235
Epoch 78/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1019.1747 - val_loss: 1175.9734
Epoch 79/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1020.1266 - val_loss: 1181.1238
Epoch 80/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1020.9978 - val_los

19948/19948 [==============================] - 0s 8us/sample - loss: 818.9702 - val_loss: 1183.2370
Epoch 145/200
19948/19948 [==============================] - 0s 7us/sample - loss: 796.2257 - val_loss: 1188.0276
Epoch 146/200
19948/19948 [==============================] - 0s 7us/sample - loss: 800.2548 - val_loss: 1191.3843
Epoch 147/200
19948/19948 [==============================] - 0s 7us/sample - loss: 787.9946 - val_loss: 1178.7354
Epoch 148/200
19948/19948 [==============================] - 0s 7us/sample - loss: 798.3653 - val_loss: 1197.1904
Epoch 149/200
19948/19948 [==============================] - 0s 8us/sample - loss: 800.5607 - val_loss: 1183.5763
Epoch 150/200
19948/19948 [==============================] - 0s 7us/sample - loss: 786.6381 - val_loss: 1193.3012
Epoch 151/200
19948/19948 [==============================] - 0s 7us/sample - loss: 784.9981 - val_loss: 1181.5430
Epoch 152/200
19948/19948 [==============================] - 0s 6us/sample - loss: 785.8560 - val_loss

In [20]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=[0])

In [21]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss=[history.history["val_loss"][-1]]
performance_summary(model, X_test, y_test )

39.70% : Share of forecasts within 5% absolute error

1189   : Mean absolute error 

10.19% : Mean absolute percentage error



## Adding Drop-out

In [22]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Dropout",)


In [23]:
model.summary()

Model: "Dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               6579

In [24]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 1s 63us/sample - loss: 11104.9790 - val_loss: 10453.7489
Epoch 2/200
19948/19948 [==============================] - 0s 8us/sample - loss: 6566.8867 - val_loss: 2844.8541
Epoch 3/200
19948/19948 [==============================] - 0s 8us/sample - loss: 2426.2825 - val_loss: 1743.4334
Epoch 4/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1758.0013 - val_loss: 1499.9646
Epoch 5/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1648.7797 - val_loss: 1414.9995
Epoch 6/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1603.1394 - val_loss: 1372.3611
Epoch 7/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1572.5695 - val_loss: 1345.1777
Epoch 8/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1534.9278 - val_loss: 1323.9781
Epoch 9/200
19948/19948 [==================

19948/19948 [==============================] - 0s 8us/sample - loss: 1344.1066 - val_loss: 1204.2787
Epoch 73/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1361.8238 - val_loss: 1212.8770
Epoch 74/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1361.7295 - val_loss: 1193.3304
Epoch 75/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1357.0677 - val_loss: 1196.8888
Epoch 76/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1368.5989 - val_loss: 1209.1847
Epoch 77/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1350.5886 - val_loss: 1218.7005
Epoch 78/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1355.7820 - val_loss: 1198.4501
Epoch 79/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1345.8968 - val_loss: 1198.2013
Epoch 80/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1362.0107 - val_los

19948/19948 [==============================] - 0s 10us/sample - loss: 1311.8990 - val_loss: 1157.2143
Epoch 144/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1291.3246 - val_loss: 1161.1286
Epoch 145/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1305.9377 - val_loss: 1172.8775
Epoch 146/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1327.5305 - val_loss: 1217.6493
Epoch 147/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1303.5474 - val_loss: 1172.1562
Epoch 148/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1304.6293 - val_loss: 1177.5680
Epoch 149/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1293.3838 - val_loss: 1181.7596
Epoch 150/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1310.2340 - val_loss: 1159.4899
Epoch 151/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1289.5568

In [25]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [26]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

38.54% : Share of forecasts within 5% absolute error

1152   : Mean absolute error 

9.82% : Mean absolute percentage error



## Adding batch normalization

In [27]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Batchnorm",)


In [28]:
model.summary()

Model: "Batchnorm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               13

In [29]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 3s 145us/sample - loss: 11193.2647 - val_loss: 11118.9992
Epoch 2/200
19948/19948 [==============================] - 0s 15us/sample - loss: 11176.2126 - val_loss: 11081.1866
Epoch 3/200
19948/19948 [==============================] - 0s 16us/sample - loss: 11140.8903 - val_loss: 11036.2253
Epoch 4/200
19948/19948 [==============================] - 0s 13us/sample - loss: 11086.9974 - val_loss: 10963.5740
Epoch 5/200
19948/19948 [==============================] - 0s 18us/sample - loss: 11010.0876 - val_loss: 10843.1675
Epoch 6/200
19948/19948 [==============================] - 0s 15us/sample - loss: 10903.1470 - val_loss: 10675.2024
Epoch 7/200
19948/19948 [==============================] - 0s 21us/sample - loss: 10759.9926 - val_loss: 10465.1814
Epoch 8/200
19948/19948 [==============================] - 0s 15us/sample - loss: 10573.1849 - val_loss: 10204.1006
Epoch 9/200
19948/1994

19948/19948 [==============================] - 0s 16us/sample - loss: 1183.5383 - val_loss: 1132.3109
Epoch 72/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1193.0288 - val_loss: 1118.6670
Epoch 73/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1172.1359 - val_loss: 1118.3066
Epoch 74/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1195.8703 - val_loss: 1115.3266
Epoch 75/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1172.0994 - val_loss: 1118.5195
Epoch 76/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1153.7113 - val_loss: 1116.5877
Epoch 77/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1158.3088 - val_loss: 1117.9578
Epoch 78/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1173.4218 - val_loss: 1121.2880
Epoch 79/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1160.5391 

Epoch 142/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1032.9513 - val_loss: 1103.5276
Epoch 143/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1011.9039 - val_loss: 1112.1450
Epoch 144/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1010.8992 - val_loss: 1102.1539
Epoch 145/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1020.2334 - val_loss: 1100.7314
Epoch 146/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1006.5899 - val_loss: 1095.9733
Epoch 147/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1018.1540 - val_loss: 1094.0600
Epoch 148/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1012.9936 - val_loss: 1121.5801
Epoch 149/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1085.6748 - val_loss: 1103.3289
Epoch 150/200
19948/19948 [==============================] - 0s 18us/sam

In [30]:
previous_val_loss[-1]

1152.263653981798

In [31]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [32]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

41.61% : Share of forecasts within 5% absolute error

1113   : Mean absolute error 

9.68% : Mean absolute percentage error



## Adding Leaky Relu

In [33]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dense(units=1, activation="linear"),



],name="LeakyRELU",)


In [34]:
model.summary()

Model: "LeakyRELU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               20

In [35]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 2s 110us/sample - loss: 11193.0127 - val_loss: 11115.5698
Epoch 2/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11179.0485 - val_loss: 11076.8315
Epoch 3/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11154.1230 - val_loss: 11051.6681
Epoch 4/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11116.2471 - val_loss: 11000.9666
Epoch 5/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11061.6192 - val_loss: 10915.5305
Epoch 6/300
19948/19948 [==============================] - 0s 17us/sample - loss: 10985.3352 - val_loss: 10820.2911
Epoch 7/300
19948/19948 [==============================] - 0s 18us/sample - loss: 10882.3317 - val_loss: 10665.9907
Epoch 8/300
19948/19948 [==============================] - 0s 18us/sample - loss: 10747.6648 - val_loss: 10502.1500
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 16us/sample - loss: 1296.2924 - val_loss: 1174.9230
Epoch 72/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1300.4500 - val_loss: 1192.4060
Epoch 73/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1285.7784 - val_loss: 1166.4650
Epoch 74/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1282.1263 - val_loss: 1171.2508
Epoch 75/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1281.6993 - val_loss: 1163.7345
Epoch 76/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1280.6535 - val_loss: 1165.4358
Epoch 77/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1277.4028 - val_loss: 1170.8390
Epoch 78/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1282.9564 - val_loss: 1163.2927
Epoch 79/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1270.4187 

Epoch 142/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1219.2089 - val_loss: 1149.0181
Epoch 143/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1220.1687 - val_loss: 1149.9302
Epoch 144/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1223.7032 - val_loss: 1140.0915
Epoch 145/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1214.2329 - val_loss: 1157.8368
Epoch 146/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1218.4128 - val_loss: 1159.5815
Epoch 147/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1229.2537 - val_loss: 1172.2902
Epoch 148/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1224.7704 - val_loss: 1138.8615
Epoch 149/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1229.3350 - val_loss: 1134.8769
Epoch 150/300
19948/19948 [==============================] - 0s 18us/sam

19948/19948 [==============================] - 0s 16us/sample - loss: 1184.2732 - val_loss: 1126.5663
Epoch 213/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1183.5091 - val_loss: 1135.6423
Epoch 214/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1184.5412 - val_loss: 1140.0388
Epoch 215/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1176.1644 - val_loss: 1122.0773
Epoch 216/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1166.1989 - val_loss: 1125.6962
Epoch 217/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1178.6928 - val_loss: 1127.5971
Epoch 218/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1178.2132 - val_loss: 1130.0447
Epoch 219/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1174.8739 - val_loss: 1120.4430
Epoch 220/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11

Epoch 283/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1151.9712 - val_loss: 1115.6588
Epoch 284/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1152.5480 - val_loss: 1124.4890
Epoch 285/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1145.2065 - val_loss: 1113.9451
Epoch 286/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1147.5380 - val_loss: 1117.9644
Epoch 287/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1141.9522 - val_loss: 1116.2165
Epoch 288/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1150.5279 - val_loss: 1112.7297
Epoch 289/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1144.3831 - val_loss: 1111.6944
Epoch 290/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1134.1284 - val_loss: 1105.6419
Epoch 291/300
19948/19948 [==============================] - 0s 17us/sam

In [36]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [37]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

40.04% : Share of forecasts within 5% absolute error

1113   : Mean absolute error 

9.49% : Mean absolute percentage error



# Expanding network architecture and learning rate tuning

## Adding hidden layer with 1024 neurons

In [38]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Larger_network",)


In [39]:
model.summary()

Model: "Larger_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              45056     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)            

In [40]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 3s 135us/sample - loss: 11191.7813 - val_loss: 11104.0071
Epoch 2/300
19948/19948 [==============================] - 0s 20us/sample - loss: 11177.7168 - val_loss: 11047.0997
Epoch 3/300
19948/19948 [==============================] - 0s 21us/sample - loss: 11149.8164 - val_loss: 10989.5884
Epoch 4/300
19948/19948 [==============================] - 0s 20us/sample - loss: 11107.0936 - val_loss: 10923.0610
Epoch 5/300
19948/19948 [==============================] - 0s 19us/sample - loss: 11046.8626 - val_loss: 10828.3631
Epoch 6/300
19948/19948 [==============================] - 0s 20us/sample - loss: 10964.7572 - val_loss: 10684.6224
Epoch 7/300
19948/19948 [==============================] - 0s 23us/sample - loss: 10855.6469 - val_loss: 10568.0095
Epoch 8/300
19948/19948 [==============================] - 0s 20us/sample - loss: 10714.1471 - val_loss: 10397.5908
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 21us/sample - loss: 1247.1129 - val_loss: 1173.2485
Epoch 72/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1243.1447 - val_loss: 1181.3267
Epoch 73/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1232.7250 - val_loss: 1156.1025
Epoch 74/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1239.1896 - val_loss: 1175.2472
Epoch 75/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1242.7366 - val_loss: 1155.6676
Epoch 76/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1232.1066 - val_loss: 1189.4137
Epoch 77/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1227.1216 - val_loss: 1169.5916
Epoch 78/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1232.6196 - val_loss: 1164.6877
Epoch 79/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1228.2829 

Epoch 142/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1153.4533 - val_loss: 1135.5223
Epoch 143/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1157.4082 - val_loss: 1123.8697
Epoch 144/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1155.8154 - val_loss: 1120.3717
Epoch 145/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1151.3753 - val_loss: 1137.0536
Epoch 146/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1157.2134 - val_loss: 1123.3693
Epoch 147/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1155.1334 - val_loss: 1122.7490
Epoch 148/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1145.1992 - val_loss: 1119.7190
Epoch 149/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1150.7928 - val_loss: 1116.8693
Epoch 150/300
19948/19948 [==============================] - 0s 16us/sam

19948/19948 [==============================] - 0s 20us/sample - loss: 1101.9900 - val_loss: 1107.6320
Epoch 213/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1102.1272 - val_loss: 1106.8279
Epoch 214/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1101.6061 - val_loss: 1110.4094
Epoch 215/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1100.9936 - val_loss: 1109.5690
Epoch 216/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1096.9917 - val_loss: 1102.2713
Epoch 217/300
19948/19948 [==============================] - ETA: 0s - loss: 1100.95 - 0s 21us/sample - loss: 1104.6567 - val_loss: 1096.3868
Epoch 218/300
19948/19948 [==============================] - 0s 23us/sample - loss: 1096.4944 - val_loss: 1099.2049
Epoch 219/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1094.7702 - val_loss: 1106.3374
Epoch 220/300
19948/19948 [==============================] -

19948/19948 [==============================] - 0s 20us/sample - loss: 1065.2670 - val_loss: 1088.4336
Epoch 283/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1067.1925 - val_loss: 1092.1124
Epoch 284/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1063.4818 - val_loss: 1099.9645
Epoch 285/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1061.9365 - val_loss: 1093.7499
Epoch 286/300
19948/19948 [==============================] - 0s 22us/sample - loss: 1056.9760 - val_loss: 1090.9625
Epoch 287/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1069.5802 - val_loss: 1092.2567
Epoch 288/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1065.8106 - val_loss: 1099.0495
Epoch 289/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1053.5089 - val_loss: 1097.3236
Epoch 290/300
19948/19948 [==============================] - 0s 19us/sample - loss: 10

In [41]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [42]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

41.33% : Share of forecasts within 5% absolute error

1091   : Mean absolute error 

9.43% : Mean absolute percentage error



## Adjusting learning rate decay

In [43]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Learning_rate_decay",)


In [44]:
# Calculating learning rate decay
learning_rate = 0.005
decay = 5e-4
n_epochs=400
n_steps_per_epoch = len(X_train) // 1024
epochs = np.arange(n_epochs)
lrs = learning_rate / (1 + decay * epochs * n_steps_per_epoch)

In [45]:
# Visualizing learning rate decay

trace0=go.Scatter(
            y=lrs,
            x=epochs,
            mode='lines',
            marker=dict(
            color="red",
            size=5,
            opacity=0.5
            )
    )
        

data=[trace0]
figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning rate decay",
                yaxis=dict(title="Learning rate"),
                xaxis=dict(title="Epoch"),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
iplot(figure)

In [46]:
#Added learning rate decay to Adam optimizer
optimizer = keras.optimizers.Adam(lr=0.005, decay=5e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=400, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/400
19948/19948 [==============================] - 3s 132us/sample - loss: 11171.3893 - val_loss: 10215.7505
Epoch 2/400
19948/19948 [==============================] - 0s 21us/sample - loss: 10988.3081 - val_loss: 7682.4615
Epoch 3/400
19948/19948 [==============================] - 0s 22us/sample - loss: 10367.4705 - val_loss: 5755.6816
Epoch 4/400
19948/19948 [==============================] - 0s 21us/sample - loss: 8943.1965 - val_loss: 8368.0323
Epoch 5/400
19948/19948 [==============================] - 0s 22us/sample - loss: 6419.0709 - val_loss: 16154.6143
Epoch 6/400
19948/19948 [==============================] - 0s 20us/sample - loss: 4153.5336 - val_loss: 10507.1267
Epoch 7/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1950.1331 - val_loss: 11215.8439
Epoch 8/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1583.2384 - val_loss: 9686.1994
Epoch 9/400
19948/19948 [======

Epoch 72/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1203.2417 - val_loss: 1173.7623
Epoch 73/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1192.2533 - val_loss: 1148.6615
Epoch 74/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1187.7563 - val_loss: 1150.8937
Epoch 75/400
19948/19948 [==============================] - 0s 24us/sample - loss: 1190.1098 - val_loss: 1156.0838
Epoch 76/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1192.0337 - val_loss: 1156.8406
Epoch 77/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1181.2613 - val_loss: 1144.4151
Epoch 78/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1184.4125 - val_loss: 1150.8255
Epoch 79/400
19948/19948 [==============================] - 0s 25us/sample - loss: 1188.5842 - val_loss: 1154.2351
Epoch 80/400
19948/19948 [==============================] - 1s 32us/sample - los

Epoch 143/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1117.7140 - val_loss: 1108.5362
Epoch 144/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1123.4595 - val_loss: 1109.4751
Epoch 145/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1119.1809 - val_loss: 1111.0172
Epoch 146/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1123.7701 - val_loss: 1104.1957
Epoch 147/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1115.8989 - val_loss: 1110.8588
Epoch 148/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1109.6549 - val_loss: 1109.1258
Epoch 149/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1118.5132 - val_loss: 1106.3682
Epoch 150/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1117.1358 - val_loss: 1107.2845
Epoch 151/400
19948/19948 [==============================] - 0s 18us/sam

19948/19948 [==============================] - 0s 20us/sample - loss: 1078.0377 - val_loss: 1087.7657
Epoch 214/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1074.1259 - val_loss: 1089.6979
Epoch 215/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1073.0438 - val_loss: 1085.8602
Epoch 216/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1069.6029 - val_loss: 1088.4962
Epoch 217/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1079.5407 - val_loss: 1092.4877
Epoch 218/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1069.6670 - val_loss: 1092.2363
Epoch 219/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1076.1975 - val_loss: 1088.8018
Epoch 220/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1071.0242 - val_loss: 1085.1493
Epoch 221/400
19948/19948 [==============================] - 0s 18us/sample - loss: 10

Epoch 284/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1055.7779 - val_loss: 1076.7997
Epoch 285/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1047.3570 - val_loss: 1076.7343
Epoch 286/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1048.9203 - val_loss: 1084.3446
Epoch 287/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1046.8679 - val_loss: 1073.3456
Epoch 288/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1050.6687 - val_loss: 1077.4098
Epoch 289/400
19948/19948 [==============================] - 1s 26us/sample - loss: 1048.5415 - val_loss: 1083.4796
Epoch 290/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1051.8813 - val_loss: 1076.3183
Epoch 291/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1053.8248 - val_loss: 1077.0233
Epoch 292/400
19948/19948 [==============================] - 1s 27us/sam

19948/19948 [==============================] - 0s 19us/sample - loss: 1039.4894 - val_loss: 1070.5929
Epoch 355/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1035.8107 - val_loss: 1072.0432
Epoch 356/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1027.9213 - val_loss: 1070.3164
Epoch 357/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1025.1526 - val_loss: 1078.2158
Epoch 358/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1024.7813 - val_loss: 1075.0656
Epoch 359/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1033.7411 - val_loss: 1071.1496
Epoch 360/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1022.8825 - val_loss: 1070.2869
Epoch 361/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1040.3852 - val_loss: 1065.6129
Epoch 362/400
19948/19948 [==============================] - 0s 17us/sample - loss: 10

In [47]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [48]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

43.05% : Share of forecasts within 5% absolute error

1068   : Mean absolute error 

9.21% : Mean absolute percentage error



## Callbacks stop

In [49]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Callbacks_stop",)


In [50]:
# Defining Callbacks
# Setting weights saving convention
checkpoint_name = 'Weights\Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
# Defining checkpoint behaviour
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [51]:
optimizer = keras.optimizers.Adam(lr=0.005, decay=5e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks_list, 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/500
19456/19948 [============================>.] - ETA: 0s - loss: 11174.9780
Epoch 00001: val_loss improved from inf to 10216.48293, saving model to Weights\Weights-001--10216.48293.hdf5
19948/19948 [==============================] - 3s 143us/sample - loss: 11171.3892 - val_loss: 10216.4829
Epoch 2/500
18432/19948 [==========================>...] - ETA: 0s - loss: 11008.7835
Epoch 00002: val_loss improved from 10216.48293 to 7990.07066, saving model to Weights\Weights-002--7990.07066.hdf5
19948/19948 [==============================] - 1s 26us/sample - loss: 10988.2583 - val_loss: 7990.0707
Epoch 3/500
17408/19948 [=========================>....] - ETA: 0s - loss: 10423.5730
Epoch 00003: val_loss improved from 7990.07066 to 6717.19043, saving model to Weights\Weights-003--6717.19043.hdf5
19948/19948 [==============================] - 1s 26us/sample - loss: 10370.8623 - val_loss: 6717.1904
Epoch 4/500
18432/19948 [================

16384/19948 [=======================>......] - ETA: 0s - loss: 1302.6645
Epoch 00030: val_loss improved from 1200.41962 to 1190.78107, saving model to Weights\Weights-030--1190.78107.hdf5
19948/19948 [==============================] - 0s 24us/sample - loss: 1299.2807 - val_loss: 1190.7811
Epoch 31/500
19456/19948 [============================>.] - ETA: 0s - loss: 1293.9886
Epoch 00031: val_loss did not improve from 1190.78107
19948/19948 [==============================] - 0s 18us/sample - loss: 1295.3961 - val_loss: 1204.6287
Epoch 32/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1290.5589
Epoch 00032: val_loss did not improve from 1190.78107
19948/19948 [==============================] - 0s 17us/sample - loss: 1291.1362 - val_loss: 1204.9877
Epoch 33/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1293.4991
Epoch 00033: val_loss did not improve from 1190.78107
19948/19948 [==============================] - 0s 17us/sample - loss: 1295.4941 - val_

19456/19948 [============================>.] - ETA: 0s - loss: 1218.9368
Epoch 00062: val_loss did not improve from 1157.38679
19948/19948 [==============================] - 0s 18us/sample - loss: 1219.9214 - val_loss: 1162.8586
Epoch 63/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1214.2023
Epoch 00063: val_loss improved from 1157.38679 to 1152.31716, saving model to Weights\Weights-063--1152.31716.hdf5
19948/19948 [==============================] - 0s 25us/sample - loss: 1212.6485 - val_loss: 1152.3172
Epoch 64/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1205.1397
Epoch 00064: val_loss did not improve from 1152.31716
19948/19948 [==============================] - 0s 17us/sample - loss: 1205.7224 - val_loss: 1160.6493
Epoch 65/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1209.9151
Epoch 00065: val_loss did not improve from 1152.31716
19948/19948 [==============================] - 0s 17us/sample - loss: 1205.7106 - val_

Epoch 00094: val_loss improved from 1135.90358 to 1135.62465, saving model to Weights\Weights-094--1135.62465.hdf5
19948/19948 [==============================] - 1s 32us/sample - loss: 1155.8324 - val_loss: 1135.6247
Epoch 95/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1159.7991
Epoch 00095: val_loss did not improve from 1135.62465
19948/19948 [==============================] - 0s 23us/sample - loss: 1157.9927 - val_loss: 1139.6051
Epoch 96/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1157.8116
Epoch 00096: val_loss improved from 1135.62465 to 1130.00532, saving model to Weights\Weights-096--1130.00532.hdf5
19948/19948 [==============================] - 1s 42us/sample - loss: 1157.1680 - val_loss: 1130.0053
Epoch 97/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1158.3897
Epoch 00097: val_loss did not improve from 1130.00532
19948/19948 [==============================] - 1s 28us/sample - loss: 1157.1595 - val_loss: 1131.1

Epoch 126/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1130.4017
Epoch 00126: val_loss did not improve from 1119.25944
19948/19948 [==============================] - 0s 19us/sample - loss: 1131.1014 - val_loss: 1122.4961
Epoch 127/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1136.3568
Epoch 00127: val_loss did not improve from 1119.25944
19948/19948 [==============================] - 0s 20us/sample - loss: 1130.9879 - val_loss: 1120.9249
Epoch 128/500
19456/19948 [============================>.] - ETA: 0s - loss: 1120.8344
Epoch 00128: val_loss did not improve from 1119.25944
19948/19948 [==============================] - 0s 18us/sample - loss: 1123.8859 - val_loss: 1120.0509
Epoch 129/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1126.3169 ETA: 0s - loss: 1
Epoch 00129: val_loss did not improve from 1119.25944
19948/19948 [==============================] - 0s 19us/sample - loss: 1121.9776 - val_loss: 1122.5373
Epoch 130/

...

Epoch 00356: val_loss improved from 1076.52419 to 1076.40146, saving model to Weights\Weights-356--1076.40146.hdf5
19948/19948 [==============================] - 1s 30us/sample - loss: 1019.3267 - val_loss: 1076.4015
Epoch 357/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1015.9570
Epoch 00357: val_loss improved from 1076.40146 to 1075.11338, saving model to Weights\Weights-357--1075.11338.hdf5
19948/19948 [==============================] - 1s 25us/sample - loss: 1015.6694 - val_loss: 1075.1134
Epoch 358/500
19456/19948 [============================>.] - ETA: 0s - loss: 1018.1465
Epoch 00358: val_loss did not improve from 1075.11338
19948/19948 [==============================] - 0s 18us/sample - loss: 1016.8924 - val_loss: 1080.1381
Epoch 359/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1021.8635
Epoch 00359: val_loss did not improve from 1075.11338
19948/19948 [==============================] - 0s 19us/sample - loss: 1022.2132 - val_loss: 107

Epoch 389/500
19456/19948 [============================>.] - ETA: 0s - loss: 1011.1725
Epoch 00389: val_loss did not improve from 1071.37012
19948/19948 [==============================] - 0s 22us/sample - loss: 1012.7996 - val_loss: 1077.5653
Epoch 390/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1009.6122
Epoch 00390: val_loss did not improve from 1071.37012
19948/19948 [==============================] - 0s 20us/sample - loss: 1007.8196 - val_loss: 1073.5157
Epoch 391/500
19456/19948 [============================>.] - ETA: 0s - loss: 1013.2810
Epoch 00391: val_loss did not improve from 1071.37012
19948/19948 [==============================] - 0s 20us/sample - loss: 1013.7762 - val_loss: 1076.5000
Epoch 392/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1013.1235
Epoch 00392: val_loss did not improve from 1071.37012
19948/19948 [==============================] - 0s 19us/sample - loss: 1011.7934 - val_loss: 1077.7070
Epoch 393/500
17408/19948 [=

16384/19948 [=======================>......] - ETA: 0s - loss: 1010.4676
Epoch 00422: val_loss did not improve from 1068.13698
19948/19948 [==============================] - 0s 18us/sample - loss: 1008.3025 - val_loss: 1071.9440
Epoch 423/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1011.5462
Epoch 00423: val_loss did not improve from 1068.13698
19948/19948 [==============================] - 0s 17us/sample - loss: 1009.3935 - val_loss: 1075.7726
Epoch 424/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1008.1476
Epoch 00424: val_loss did not improve from 1068.13698
19948/19948 [==============================] - 0s 17us/sample - loss: 1008.4260 - val_loss: 1074.5454
Epoch 425/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1002.9310
Epoch 00425: val_loss did not improve from 1068.13698
19948/19948 [==============================] - 0s 19us/sample - loss: 1005.3234 - val_loss: 1078.4584
Epoch 426/500
19456/19948 [===============

Epoch 456/500
19456/19948 [============================>.] - ETA: 0s - loss: 998.0949
Epoch 00456: val_loss did not improve from 1067.48982
19948/19948 [==============================] - 0s 22us/sample - loss: 997.2907 - val_loss: 1069.7882
Epoch 457/500
17408/19948 [=========================>....] - ETA: 0s - loss: 991.8743
Epoch 00457: val_loss did not improve from 1067.48982
19948/19948 [==============================] - 0s 23us/sample - loss: 991.4254 - val_loss: 1068.9737
Epoch 458/500
19456/19948 [============================>.] - ETA: 0s - loss: 993.0663
Epoch 00458: val_loss did not improve from 1067.48982
19948/19948 [==============================] - 0s 20us/sample - loss: 994.1642 - val_loss: 1068.8588
Epoch 459/500
17408/19948 [=========================>....] - ETA: 0s - loss: 995.9641
Epoch 00459: val_loss did not improve from 1067.48982
19948/19948 [==============================] - 1s 26us/sample - loss: 996.7141 - val_loss: 1069.0852
Epoch 460/500
19456/19948 [=========

17408/19948 [=========================>....] - ETA: 0s - loss: 992.1124
Epoch 00489: val_loss did not improve from 1066.94494
19948/19948 [==============================] - 0s 19us/sample - loss: 994.7424 - val_loss: 1071.5131
Epoch 490/500
17408/19948 [=========================>....] - ETA: 0s - loss: 994.4136
Epoch 00490: val_loss did not improve from 1066.94494
19948/19948 [==============================] - 0s 21us/sample - loss: 993.3577 - val_loss: 1067.8464
Epoch 491/500
19456/19948 [============================>.] - ETA: 0s - loss: 988.0432
Epoch 00491: val_loss improved from 1066.94494 to 1066.71914, saving model to Weights\Weights-491--1066.71914.hdf5
19948/19948 [==============================] - 1s 29us/sample - loss: 987.2654 - val_loss: 1066.7191
Epoch 492/500
18432/19948 [==========================>...] - ETA: 0s - loss: 998.4959 
Epoch 00492: val_loss did not improve from 1066.71914
19948/19948 [==============================] - 0s 21us/sample - loss: 997.6327 - val_loss

In [52]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [53]:
# Choosing most recent weights file - as only best epochs are saved it will 
# be equivalent to lowest error
list_of_files = glob.glob('Weights/*') 
latest_file = max(list_of_files, key=os.path.getmtime)

In [54]:
# Loading best weights and compiling model
weights_file =  latest_file 
model.load_weights(weights_file) 
model.compile(loss='mean_absolute_error', optimizer='adam')

In [55]:
# Storing model performance as reference for tracking progress, evaluating key KPIs
previous_val_loss.append(np.asarray(history.history["val_loss"]).min())
performance_summary(model, X_test, y_test )

42.87% : Share of forecasts within 5% absolute error

1067   : Mean absolute error 

9.13% : Mean absolute percentage error



# Summary

In [57]:
# Combining validation loss with model names
models=["basic_model","dropout","batch_norm","leakyRELU","1024_layer","lr_decay","callbacks"]
df_comparison=pd.DataFrame(previous_val_loss,columns=["val_loss"])
df_comparison["model"]=models

In [58]:
# Plotting model evolution
import plotly.express as px

fig = px.line(df_comparison, x="model", y="val_loss")
fig.show()